In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# for utils
import scipy
from scipy.spatial.transform import Rotation as Rot

In [ ]:
# utils
def plot_poses(Trs, enum=False):
    xs = []
    ys = []
    plt.plot(0, 0, marker='o', markersize=20, c='r', linestyle='None', label='Origin')
    for index, Tr in enumerate(Trs):
        x = Tr[0,2]
        y = Tr[1,2]
        R3 = np.eye(3)
        R3[:2,:2] = Tr[:2,:2]
        t = Rot.from_matrix(R3).as_rotvec()[2]
        
        plt.plot(x, y, marker=(3, 0, 180*t/np.pi-90), markersize=20, linestyle='None')
        plt.plot(x, y, marker=(2, 0, 180*t/np.pi-90), markersize=30, linestyle='None')
        if enum:
            plt.text(x, y, index, size=20)
        xs.append(x)
        ys.append(y)
    plt.plot(xs, ys)    
    plt.legend()
    plt.axis('equal')
    plt.grid()
    
def gTl_is_correct():
    x_ = np.random.randn(3)
    x_[2] = np.clip(x_[2], -np.pi/2.1, +np.pi/2.1)
    res = gTl(x_)
    assert res.shape == (3,3), "T must be 3x3 matrix"
    
    mat = np.eye(3); mat[0,2] = x_[0]; mat[1,2] = x_[1]
    if 'as_matrix' in dir(Rot):
        mat[:2,:2] = Rot.from_rotvec([0,0,x_[2]]).as_matrix()[:2,:2]
    else:
        mat[:2,:2] = Rot.from_rotvec([0,0,x_[2]]).as_dcm()[:2,:2]
    return np.linalg.norm(np.linalg.inv(mat) @ res - np.eye(3)) < 1e-14

# I. Motion model

This is a two-wheeled robot.
<img src="./motion.png">

The robot has the following motion model.  

$\begin{bmatrix} x\\y\\ \theta \end{bmatrix}_t = \begin{bmatrix} x\\y\\ \theta \end{bmatrix}_{t-1} + \Delta t
\begin{bmatrix} 
  \frac{1}{2} \, \cos \theta_{t-1} & \frac{1}{2} \, \cos \theta_{t-1} \\ 
  \frac{1}{2} \, \sin \theta_{t-1} & \frac{1}{2} \, \sin \theta_{t-1} \\ 
  \frac{1}{l} & -\frac{1}{l} 
\end{bmatrix}
\begin{bmatrix} v_1 + \eta_{v_1}\\ v_2 + \eta_{v_2}\end{bmatrix}_t + \begin{bmatrix}\eta_x \\ \eta_y \\ \eta_\theta \\ \end{bmatrix}$

Notice, that here Gaussian noise is presented in both: _state_ and _action_ space. $\eta_{\mathrm x} \sim \mathcal N (0, R)$, $\eta_{u} \sim \mathcal N (0, M)$

It is almost the same model as in the lecture 5 but with a little different action vector
$u = \begin{bmatrix} v_1 \\ v_2 \end{bmatrix}$

<br/>
<br/>

Linearization of transition function is needed to feed this model to (Extended) Kalman filter:

$x_t = g(\mathrm x_{t-1}, u_t) \simeq g(\mathrm \mu_{t-1}, \bar u_t) + \left.\frac{\partial g(x_{t-1},u_t)} { \partial x_{t-1}} \right|_{\mu_{t-1}} \cdot (x_{t-1}-\mu_{t-1}) + \left.\frac{\partial g(x_{t-1},u_t)} {\partial u_t} \right|_{\bar u_t} \cdot (u_t-\bar u_t)$

$G_t= \left.\frac{\partial g(x_{t-1},u_t)} { \partial x_{t-1}} \right|_{\mu_{t-1}} $  

$V_t= \left.\frac{\partial g(x_{t-1},u_t)} {\partial u_t} \right|_{\overline u_t}$    

### Task.

Obtain $G_t$, $V_t$.



_Hint. In practice you can forget about noise terms while linearising:_

$\begin{bmatrix} x\\y\\ \theta \end{bmatrix}_t = \begin{bmatrix} x\\y\\ \theta \end{bmatrix}_{t-1} + \Delta t
\begin{bmatrix} 
  \frac{1}{2} \, \cos \theta_{t-1} & \frac{1}{2} \, \cos \theta_{t-1} \\ 
  \frac{1}{2} \, \sin \theta_{t-1} & \frac{1}{2} \, \sin \theta_{t-1} \\ 
  \frac{1}{l} & -\frac{1}{l} 
\end{bmatrix}
\begin{bmatrix} v_1\\ v_2\end{bmatrix}_t$

_Hint for $G_t$. You may do matrix multiplication of action part beforehand to get simpler expression:  
$B \cdot u$ will have dims 3x1 instead of 3x2 plus 2x1._

You can write answer here below or handwrite.

$G_t = 
%Fill in G_t below
\begin{bmatrix}
  ? & ? & ?\\
  ? & ? & ?\\
  ? & ? & ?
\end{bmatrix}
$

$V_t = 
%Fill in F_t below
?
$

# II. RBT

According to Lynch book [_Lynch, Kevin M., and Frank C. Park. Modern Robotics. Cambridge University Press, 2017._
]  
there are three major uses for a transformation matrix $T$.  

__Use (a): to represent the configuration (position and orientation) of a rigid body;__

Suppose, the origin of a robot (= origin of robot's ref. frame) is located in the point $(x,y)$ of a global frame   
and robot's frame is rotated by angle $\theta$:

<img src="./global_local.png">

Its pose (pose = position + orientation) can be represented by the vector $$\mathrm x = \begin{bmatrix} x\\y\\ \theta \end{bmatrix}$$

or by the transformation matrix:
$$^gT_l(\mathrm x) = \operatorname{some\_mapping}(\mathrm x)$$

_Here $^gT_l$ means that local frame is described wrt. global frame_

### Task 1. (already solved for you)
Code $\operatorname{some\_mapping}(\mathrm x)$

In [ ]:
# T(x)
# use numpy only
def gTl(state):
    def c(t):
        return np.cos(t)
    def s(t):
        return np.sin(t)
    x, y, t = state
    return np.array([
        [c(t), -s(t), x],
        [s(t), c(t), y],
        [0, 0, 1]
    ])

In [ ]:
# Check out
gTl_is_correct()

In [ ]:
# Robot at the origin of global frame
x = [0,0,0]
T_origin = gTl(x)
T_origin

In [ ]:
# plot
Ts = [T_origin]
plot_poses(Ts)

In [ ]:
# Purely rotated robot
x = [0,0,np.pi/6]
T_rot = gTl(x)
T_rot

In [ ]:
# plot
Ts = [T_rot]
plot_poses(Ts)

In [ ]:
# Purely translated robot
x = [5,4,0]
T_transl = gTl(x)
T_transl

In [ ]:
# plot
Ts = [T_transl]
plot_poses(Ts)

<br/>
<br/>

Sometimes there is need to describe global frame wrt. local frame.  
Imagine robot computing the way back home that is located in the origin of global frame.  

<img src="./inverse.png">

This can be simply done by inverse transformation $$^lT_g = (^gT_l)^{-1}.$$

### Task 2.
This is a theoretical task. Obtain and write down the inverse transformation.
Use the internal structure of the transformation matrix:
$$^gT_l = \begin{bmatrix} R & t \\ 0^\top & 1 \end{bmatrix},$$
where the rotation matrix $R$ and displacement $t$ are known.

_Trick: better to:_
- do block multiplication of $^gT_l \, ^lT_g  = I$:
$$\begin{bmatrix} R & t \\ 0^\top & 1 \end{bmatrix} \cdot \begin{bmatrix} M & a \\ 0^\top & 1 \end{bmatrix} = I$$
- express unknown $M$ and $a$ through $R$ and $t$
- assemble $T^{-1}$ by $M$ and $a$

__$^gT_l^{-1}(R,t)$ must not have any matrix inverse inside!__

_Your markdown here_

<br/>
<br/>
<br/>

__Use (b): to change the reference frame in which a vector or frame is represented;__  

Suppose, the robot observes some landmark $p$ in position $^lp = \begin{bmatrix} ^l x \\ ^l y \\ 1 \end{bmatrix}$ of __robot's__ frame.  
The robot must send global coordinates $^gp$ of $p$ to computational module that process global coordinates of landmarks and plots a map of landmarks. 
<img src="./vector.png">

### Task 3.A.

Write down the formula to get $^gp$ with known $^lp$ and robot's pose $^gT_l$.  
Provide numerical result for given $^lp$ and $^gT_l$.

In [ ]:
# Given data
lp = [1.0, 1.3, 1]
T = gTl([0.4, 0.5, 1.2])

# Your code here


__Use (c): to displace a vector or frame.__  

Suppose, the robot has current pose $^gT_a$ wrt. global frame.  
Then the robot updates its pose by $^aT_b$ and the updated state is $^gT_b$.
<img src="./delta.png">

### Task 3.B.
Obtain $^gT_b$ given pose $a$ and transformation from $a$ to $b$.

In [ ]:
state_a = [0.3,0.5,0]
atob = [0.3, -0.4, -np.pi/6]

In [ ]:
gTa = gTl(state_a)
gTa

In [ ]:
# plot
Ts = [gTa]
plot_poses(Ts)

In [ ]:
aTb = gTl(atob)
aTb

In [ ]:
# Your code here
gTb = 
gTb

In [ ]:
# plot
Ts = [gTa, gTb]
plot_poses(Ts)

In addition, uses (a), (b), (c) can be applied simultaneously due to identiacal mathematical description of them.  
For instance, (b) and (c) are shown in the lecture 5.